In [4]:
import autograd.numpy as np 
from autograd import grad
from scipy import optimize
import zipfile
import pandas as pd

In [12]:
class param_adj:
    def __init__(self, V_data, t_data, I_data, dt, init_guess, stoch_var = [ 0.05, 0.6, 0.32], bounds = [], method = 'BFGS', tol = 1e-5):
        
        #variables from upload.py
        self.V0 = V_data[0]
        self.V_data = V_data
        self.I_data = I_data
        self.t_data = t_data
        self.m = stoch_var[0]
        self.h = stoch_var[1]
        self.n = stoch_var[2]
        self.init_guess = init_guess
        
        #simulation parameters
        self.dt = dt
        self.t_sim = np.arange(0, t_data[-1], dt)
        
        #set optimization parameters
        self.bounds = bounds
        self.method = method
        self.tol = tol
    
    # Define the HH model helper equations, note these are repeated from the stim_adj class, but for independent completeness included seperately
    def alpha_m(self, V):
        '''transition rate constant for m-gates (rapid response Na) shut gates opening as a function of voltage'''
        return 0.1 * (V + 40.0) / (1.0 - np.exp(-(V + 40.0) / 10.0))

    def beta_m(self, V):
        '''transition rate constant for m-gates (rapid response Na) open gates closing as a function of voltage '''
        return 4.0 * np.exp(-(V + 65.0) / 18.0)

    def alpha_h(self, V):
        '''transition rate constant for h-gates (slow response Na) shut gates opening as a function of voltage'''
        return 0.07 * np.exp(-(V + 65.0) / 20.0)

    def beta_h(self, V):
        '''transition rate constant for h-gates (slow response Na) open gates closing as a function of voltage '''
        return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))

    def alpha_n(self, V):
        '''transition rate constant for n-gates (slow response K) shut gates opening as a function of voltage'''
        return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))

    def beta_n(self, V):
        '''transition rate constant for n-gates (slow response K) open gates closing as a function of voltage '''
        return 0.125 * np.exp(-(V + 65) / 80.0)
    
    def __forward(self, params, I, V, t, m, n, h):
        g_Na, g_K, g_L, E_Na, E_K, E_L, C_m = params
        dVdt = (I - g_Na * m**3 * h * (V - E_Na) - g_K * n**4 * (V - E_K) - g_L * (V - E_L)) / C_m
        dmdt = self.alpha_m(V) * (1 - m) - self.beta_m(V) * m
        dhdt = self.alpha_h(V) * (1 - h) - self.beta_h(V) * h
        dndt = self.alpha_n(V) * (1 - n) - self.beta_n(V) * n
        return dVdt, dmdt, dhdt, dndt
    
    def integrate_HH(self, params):
        g_Na, g_K, g_L, E_Na, E_K, E_L, C_m = params
        V_record = np.zeros_like(self.t_sim)
        V = self.V0
        
        m = self.m
        n = self.n
        h = self.h 
        
        for i in range(len(self.t)):
            V_record[i] = V
            dVdt, dmdt, dhdt, dndt =self.__forward(params, self.I_data[i], V, self.t_sim[i], m, n, h)
            V += dVdt * self.dt
            m += dmdt * self.dt
            h += dhdt * self.dt
            n += dndt * self.dt
        return V_record

        
    def __cost(self, params): 
        cost = 0
        
        V_record = []
        V = self.V0
        g_Na, g_K, g_L, E_Na, E_K, E_L, C_m = params
        
        #initialize m, n, h
        m = self.m
        n = self.n
        h = self.h 
        
        for i in range(len(self.t_sim)):
        
            # run forward step
            V_record.append(V)
        
            dVdt, dmdt, dhdt, dndt = self.__forward(params, self.I_data[i], V, self.t_sim[i], m, n, h)
            V += dVdt * self.dt
            m += dmdt * self.dt
            h += dhdt * self.dt
            n += dndt * self.dt

            # compute cost
            if self.t_sim[i] in t_data:
                j = np.where(self.t_data == self.t_sim[i])
                cost += (V_record[i] - self.V_data[j])**2
         
        cost = cost/len(self.t_data)

            
        return cost
    
    def optimize(self): 
        grad_AD = grad(self.__cost, 0)
        optim = optimize.minimize(self.__cost, self.init_guess, args = (), jac = grad_AD, bounds = self.bounds, method = self.method, tol = self.tol)
        return optim
    

Testing the above class: 

In [13]:
zip_file_path = './sim_data/gt_1a_100.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Assume there is only one CSV file in the zip (you might need to modify this if there are multiple CSV files)
    csv_file_name = zip_ref.namelist()[0]

    # Read the CSV file directly from the zip file into a pandas DataFrame
    with zip_ref.open(csv_file_name) as csv_file:
        df = pd.read_csv(csv_file)
        

I_data = df['stim'].to_numpy()
V_data = df['voltage'].to_numpy()
t_data = df['time'].to_numpy()
dt = df['time'][1]-df['time'][0]
dur =(df['time'].to_numpy())[-1]
t = np.arange(0, dur+dt, dt)
init_guess = [123.0, 36.0, 0.4, 52.0, -72.0, -54.0, 1.0]
bnds = [(2, 180), (10, 40), (0.1, 0.5), (30, 100), (-100, -50), (-120, -50), (.9, 1.0)]


In [14]:
instance = param_adj(V_data, t_data, I_data, dt, init_guess)
instance.optimize()

/Users/sophiaepst/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:569: RuntimeWarning: Method BFGS cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/Users/sophiaepst/anaconda3/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
/var/folders/y5/qy3mtncn089941sxdcnsvzn80000gn/T/ipykernel_29354/742790953.py:51: RuntimeWarning: invalid value encountered in scalar multiply
  dmdt = self.alpha_m(V) * (1 - m) - self.beta_m(V) * m
/var/folders/y5/qy3mtncn089941sxdcnsvzn80000gn/T/ipykernel_29354/742790953.py:52: RuntimeWarning: invalid value encountered in scalar multiply
  dhdt = self.alpha_h(V) * (1 - h) - self.beta_h(V) * h
/var/folders/y5/qy3mtncn089941sxdcnsvzn80000gn/T/ipykernel_29354/742790953.py:53: RuntimeWarning: invalid value encountered in scalar multiply
  dndt = self.alpha_n(V) * (1 - n) - self.beta_n(V) * n
/Users/sophiaepst/anaconda3/lib/python3.11/site-packages

/Users/sophiaepst/anaconda3/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
/Users/sophiaepst/anaconda3/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in subtract
  return f_raw(*args, **kwargs)
/Users/sophiaepst/anaconda3/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in multiply
  return f_raw(*args, **kwargs)


  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: nan
        x: [-1.657e+02  1.094e+03 -1.877e+03 -1.683e+02 -2.121e+03
             1.862e+03  2.758e+03]
      nit: 3
      jac: [       nan        nan        nan        nan        nan
                   nan        nan]
 hess_inv: [[ 1.002e+00 -7.055e-03 ... -3.317e-03 -3.993e-02]
            [-7.055e-03  1.025e+00 ...  5.797e-03  1.564e-01]
            ...
            [-3.317e-03  5.797e-03 ...  7.873e-01  5.628e-01]
            [-3.993e-02  1.564e-01 ...  5.628e-01  6.738e-01]]
     nfev: 122
     njev: 122